# Topic Modeling and Latent Dirichlet Allocation (LDA) in Python

Source: https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [22]:
# 1. The Data

import pandas as pd

data = pd.read_csv('data/ABCNews/abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]

data_text['index'] = data_text.index
documents = data_text

print(len(documents))
print('\n\n')
print(documents[:5])

1226258



                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [21]:
# 2. Data Pre-processing

# Loading gensim and nltk libraries
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

stemmer = PorterStemmer()

# Write a function to perform lemmatize and stem preprocessing steps on the data set.
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

# Select a document to preview after preprocessing.
doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)

print('\n\ntokenized and lemmatized document: ')
print(preprocess(doc_sample))

# Preprocess the headline text, saving the results as ‘processed_docs’
processed_docs = documents['headline_text'].map(preprocess)
print('\n\n')
print(processed_docs[:10])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tonja\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


tokenized and lemmatized document: 
['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']



0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object


In [24]:
# 3. Bag of Words on the Data set

dictionary = gensim.corpora.Dictionary(processed_docs)

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break
        
# Filter out tokens that appear in
#   - less than 15 documents (absolute number) or
#   - more than 0.5 documents (fraction of total corpus size, not absolute number).
#   - after the above two steps, keep only the first 100000 most frequent tokens.

dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

# Gensim doc2bow
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
print('\n\n')
print(bow_corpus[4310])

# Preview Bag Of Words for our sample preprocessed document.
bow_doc_4310 = bow_corpus[4310]

print('\n\n')
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 

bow_doc_4310[i][1]))

# TF-IDF
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

from pprint import pprint

print('\n\n')
for doc in corpus_tfidf:
    pprint(doc)
    break

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit



[(162, 1), (240, 1), (292, 1), (589, 1), (839, 1), (3578, 1), (3579, 1)]



Word 162 ("govt") appears 1 time.
Word 240 ("group") appears 1 time.
Word 292 ("vote") appears 1 time.
Word 589 ("local") appears 1 time.
Word 839 ("want") appears 1 time.
Word 3578 ("compulsori") appears 1 time.
Word 3579 ("ratepay") appears 1 time.



[(0, 0.5852942020878993),
 (1, 0.38405854933668493),
 (2, 0.5017732999224691),
 (3, 0.5080878695349914)]


In [25]:
# 4. Running LDA using Bag of Words

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.028*"donald" + 0.025*"kill" + 0.023*"market" + 0.022*"attack" + 0.019*"border" + 0.014*"morrison" + 0.014*"dead" + 0.013*"interview" + 0.012*"bank" + 0.010*"find"
Topic: 1 
Words: 0.033*"sydney" + 0.031*"polic" + 0.029*"year" + 0.026*"news" + 0.022*"record" + 0.021*"hous" + 0.017*"rise" + 0.017*"investig" + 0.016*"death" + 0.013*"shoot"
Topic: 2 
Words: 0.019*"miss" + 0.018*"work" + 0.016*"speak" + 0.015*"countri" + 0.014*"victim" + 0.014*"releas" + 0.014*"australia" + 0.013*"china" + 0.013*"season" + 0.012*"search"
Topic: 3 
Words: 0.020*"women" + 0.018*"feder" + 0.017*"peopl" + 0.016*"health" + 0.016*"farmer" + 0.014*"win" + 0.014*"labor" + 0.013*"take" + 0.012*"servic" + 0.011*"beat"
Topic: 4 
Words: 0.024*"bushfir" + 0.023*"school" + 0.021*"canberra" + 0.018*"brisban" + 0.017*"busi" + 0.014*"student" + 0.011*"game" + 0.010*"hotel" + 0.009*"darwin" + 0.009*"port"
Topic: 5 
Words: 0.036*"elect" + 0.026*"live" + 0.016*"island" + 0.014*"china" + 0.012*"state" + 0.010

In [26]:
# 5. Running LDA using TF-IDF

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.011*"final" + 0.009*"world" + 0.009*"australia" + 0.008*"friday" + 0.008*"thursday" + 0.007*"leagu" + 0.007*"david" + 0.006*"open" + 0.006*"win" + 0.006*"cricket"
Topic: 1 Word: 0.010*"monday" + 0.007*"coronaviru" + 0.007*"alan" + 0.006*"univers" + 0.006*"zealand" + 0.006*"airport" + 0.006*"jam" + 0.006*"quarantin" + 0.006*"april" + 0.006*"rais"
Topic: 2 Word: 0.025*"news" + 0.016*"rural" + 0.009*"nation" + 0.009*"violenc" + 0.009*"climat" + 0.008*"turnbul" + 0.007*"vaccin" + 0.007*"march" + 0.007*"social" + 0.007*"domest"
Topic: 3 Word: 0.021*"coronaviru" + 0.015*"covid" + 0.011*"queensland" + 0.011*"market" + 0.010*"live" + 0.008*"australia" + 0.007*"morrison" + 0.007*"victoria" + 0.007*"australian" + 0.007*"farmer"
Topic: 4 Word: 0.032*"trump" + 0.019*"donald" + 0.013*"interview" + 0.010*"scott" + 0.009*"royal" + 0.009*"john" + 0.008*"christma" + 0.008*"michael" + 0.007*"lockdown" + 0.007*"coronaviru"
Topic: 5 Word: 0.011*"govern" + 0.009*"health" + 0.007*"care" + 0

In [28]:
# 6. Performance evaluation by classifying sample document using LDA Bag of Words model

processed_docs[4310]

for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8874749541282654	 
Topic: 0.024*"govern" + 0.019*"crash" + 0.017*"chang" + 0.015*"restrict" + 0.014*"hospit" + 0.014*"plan" + 0.012*"region" + 0.012*"council" + 0.011*"fund" + 0.011*"nation"

Score: 0.012506323866546154	 
Topic: 0.036*"elect" + 0.026*"live" + 0.016*"island" + 0.014*"china" + 0.012*"state" + 0.010*"say" + 0.010*"john" + 0.009*"emerg" + 0.009*"liber" + 0.009*"parti"

Score: 0.01250259205698967	 
Topic: 0.047*"australian" + 0.042*"trump" + 0.012*"fight" + 0.012*"royal" + 0.011*"scott" + 0.011*"say" + 0.010*"street" + 0.010*"drum" + 0.010*"australia" + 0.010*"travel"

Score: 0.012502418830990791	 
Topic: 0.024*"bushfir" + 0.023*"school" + 0.021*"canberra" + 0.018*"brisban" + 0.017*"busi" + 0.014*"student" + 0.011*"game" + 0.010*"hotel" + 0.009*"darwin" + 0.009*"port"

Score: 0.012502401135861874	 
Topic: 0.027*"case" + 0.027*"charg" + 0.026*"court" + 0.022*"murder" + 0.021*"polic" + 0.018*"woman" + 0.017*"face" + 0.014*"test" + 0.014*"jail" + 0.013*"trial"

Score

In [29]:
# 7. Performance evaluation by classifying sample document using LDA TF-IDF model.

for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.8874776363372803	 
Topic: 0.011*"govern" + 0.009*"health" + 0.007*"care" + 0.006*"fund" + 0.006*"feder" + 0.006*"updat" + 0.006*"sport" + 0.006*"coronaviru" + 0.006*"elect" + 0.006*"mental"

Score: 0.012503174133598804	 
Topic: 0.010*"andrew" + 0.007*"explain" + 0.007*"daniel" + 0.006*"august" + 0.006*"georg" + 0.006*"territori" + 0.005*"say" + 0.005*"toni" + 0.005*"ash" + 0.005*"northern"

Score: 0.012502851895987988	 
Topic: 0.021*"coronaviru" + 0.015*"covid" + 0.011*"queensland" + 0.011*"market" + 0.010*"live" + 0.008*"australia" + 0.007*"morrison" + 0.007*"victoria" + 0.007*"australian" + 0.007*"farmer"

Score: 0.012502751313149929	 
Topic: 0.010*"monday" + 0.007*"coronaviru" + 0.007*"alan" + 0.006*"univers" + 0.006*"zealand" + 0.006*"airport" + 0.006*"jam" + 0.006*"quarantin" + 0.006*"april" + 0.006*"rais"

Score: 0.01250250730663538	 
Topic: 0.019*"countri" + 0.013*"hour" + 0.008*"juli" + 0.007*"bushfir" + 0.007*"coal" + 0.006*"june" + 0.006*"busi" + 0.006*"energi" + 0.

In [31]:
# 8. Testing model on unseen document

unseen_document = 'How a Pentagon deal became an identity crisis for Google'

bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.4942493438720703	 Topic: 0.019*"miss" + 0.018*"work" + 0.016*"speak" + 0.015*"countri" + 0.014*"victim"
Score: 0.20574401319026947	 Topic: 0.047*"australian" + 0.042*"trump" + 0.012*"fight" + 0.012*"royal" + 0.011*"scott"
Score: 0.1833079755306244	 Topic: 0.024*"bushfir" + 0.023*"school" + 0.021*"canberra" + 0.018*"brisban" + 0.017*"busi"
Score: 0.01667345128953457	 Topic: 0.020*"women" + 0.018*"feder" + 0.017*"peopl" + 0.016*"health" + 0.016*"farmer"
Score: 0.01667131297290325	 Topic: 0.024*"govern" + 0.019*"crash" + 0.017*"chang" + 0.015*"restrict" + 0.014*"hospit"
Score: 0.016671018674969673	 Topic: 0.033*"sydney" + 0.031*"polic" + 0.029*"year" + 0.026*"news" + 0.022*"record"
Score: 0.016670983284711838	 Topic: 0.063*"coronaviru" + 0.042*"australia" + 0.033*"queensland" + 0.031*"covid" + 0.026*"victoria"
Score: 0.016670681536197662	 Topic: 0.036*"elect" + 0.026*"live" + 0.016*"island" + 0.014*"china" + 0.012*"state"
Score: 0.01667061634361744	 Topic: 0.028*"donald" + 0.025*